In [ ]:
import sys
import os
# Append the library path to PYTHONPATH, so library can be imported.
sys.path.append(os.path.dirname(os.getcwd()))

import shutil
import datetime
import numpy as np
import pandas as pd

from library import common as cm
from library import regression_aux as raux

In [ ]:
%run setup.py
%matplotlib inline

np.random.seed(RANDOM_SEED)

In [ ]:
%run Load_Clean_aux.py
max_period = max([int(s[6:]) for s in df.columns if 'period' in s])

In [ ]:
def save_tables(res, dirs):
    if res is not None:
        res['df_coef'].to_csv(f'{dirs}coef.csv')
        res['df_fit_std'].to_csv(f'{dirs}std.csv')

### NO permute

In [ ]:
PERMUTE = False

for VIX in [False, True]:
    
    res_dir = f"{DATA_DIR}Result/FREQ={FREQ}_HALFMONEY={HALF_MONEY}_MINM={MIN_M}_MAXM={MAX_M}_MINTAU={MIN_TAU_Days}_Permute={PERMUTE}_VIX={VIX}_MISSING={FEED_MISSING}_WINDOW={date_window}_KickTmrZeroVolume={REMOVE_TMR_ZERO_VOLUME}/"
    regr_dir = res_dir + 'Regression/'
    if os.path.exists(regr_dir):
        shutil.rmtree(regr_dir)
    os.makedirs(regr_dir)
    shutil.copy('setup.py', regr_dir);
    
    # Baseline: Black-Scholes 
    sub_regr_dir = f'{regr_dir}BS_Benchmark/'
    os.makedirs(f'{sub_regr_dir}pnl/', exist_ok=True)
    pnl_path = f'{sub_regr_dir}pnl/pnl.csv'
    
    cm.store_pnl(df, df['delta_bs'], pnl_path)
    save_tables(None, sub_regr_dir)
    
    # LR
    sub_regr_dir = f'{regr_dir}DVV/'
    os.makedirs(f'{sub_regr_dir}pnl/', exist_ok=True)
    kwargs = {
        'vix': VIX,
        'features': ['delta_bs', 'vega_n', 'vanna_n'],
        'max_period': max_period,
        'sub_res': f'{sub_regr_dir}pnl/',
        'pnl_path': f'{sub_regr_dir}pnl/pnl.csv',
        'df': df,
        'delta_coeff_1': False
    }
    res = raux.run_store_lin(**kwargs)
    save_tables(res, sub_regr_dir)  
    

### Permute

In [ ]:
PERMUTE = True

for VIX in [False, True]:
    
    res_dir = f"{DATA_DIR}Result/FREQ={FREQ}_HALFMONEY={HALF_MONEY}_MINM={MIN_M}_MAXM={MAX_M}_MINTAU={MIN_TAU_Days}_Permute={PERMUTE}_VIX={VIX}_MISSING={FEED_MISSING}_WINDOW={date_window}_KickTmrZeroVolume={REMOVE_TMR_ZERO_VOLUME}/"
    regr_dir = res_dir + 'Regression/'
    if os.path.exists(regr_dir):
        shutil.rmtree(regr_dir)
    os.makedirs(regr_dir)
    shutil.copy('setup.py', regr_dir);
    
    # BS
    df_permute = df.copy()
    for i in range(NUM_PERMUTE):
        df_permute = cm.rolling_permute(df_permute, random_seed=i + 100)
        sub_res = f'{regr_dir}BS_Benchmark/pnl/'
        os.makedirs(sub_res, exist_ok=True)
        pnl_path = f'{sub_res}pnl{i}.csv'
        cm.store_pnl(df_permute, df_permute['delta_bs'], pnl_path)
    del df_permute
    
    # LR
    df_permute = df.copy()
    sub_regr_dir = f'{regr_dir}DVV/'
    for i in range(NUM_PERMUTE):
        df_permute = cm.rolling_permute(df_permute, random_seed=i + 100)
        kwargs = {'vix': VIX, 
              'features': ['delta_bs', 'vega_n', 'vanna_n'], 
              'max_period': max_period, 
              'sub_res': f'{sub_regr_dir}pnl/',
              'pnl_path': f'{sub_regr_dir}pnl/pnl{i}.csv',
              'df': df_permute,
             'delta_coeff_1': False,
                 'agg_side': False}
        
        res = raux.run_store_lin(**kwargs)